In [8]:
import os
import sys
from pymongo import MongoClient
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()

sys.path.append('/Users/issa/Desktop/moodlyze/data')

from cleaning.cleaning import clean_text, tokenize


# Définir les chemins
dataset_path = os.path.join(os.path.expanduser("~/Desktop/moodlyze/data"), "raw", "raw_tweets.csv")
processed_dir = os.path.join(os.path.expanduser("~/Desktop/moodlyze/data"), "processed")
cleaned_dataset_path = os.path.join(processed_dir, "cleaned_tweets.csv")

# Charger le jeu de données
df = pd.read_csv(dataset_path, encoding='latin-1')
df.columns = ['number', 'id', 'date', 'query', 'user', 'text']
df = df.drop(['number', 'query'], axis=1)

# Vérifier si le fichier nettoyé existe déjà
if not os.path.exists(cleaned_dataset_path):
    tqdm.pandas()  # Initialiser la barre de progression

    tqdm.pandas(desc="Cleaning text")
    df['cleaned_text'] = df['text'].progress_apply(clean_text)
    df_subset = df.head(1000)


    # df['tokens'] = df['cleaned_text'].apply(tokenize)

    MONGODB_URL = os.getenv('MONGODB_URL')
    DB_NAME = os.getenv('DB_NAME')

    client = MongoClient(MONGODB_URL)
    db = client[DB_NAME]
    collection = db['tweets']
    data = df.to_dict(orient='records')
    collection.insert_many(data)

    print(f"Données insérées dans MongoDB sous la collection : {collection.name}")

    for doc in collection.find().limit(5):
        print(doc)



Cleaning text: 100%|██████████| 1599999/1599999 [02:04<00:00, 12836.57it/s]


OperationFailure: you are over your space quota, using 520 MB of 512 MB, full error: {'ok': 0, 'errmsg': 'you are over your space quota, using 520 MB of 512 MB', 'code': 8000, 'codeName': 'AtlasError'}